In [1]:
import pymatgen
import numpy as np
from mp_api.client import MPRester

import pandas as pd
import re
import chemparse


In [2]:
# Yoink all binary oxides from the materials project database. Find oxides with the lowest formation energy.
api_key = 'Lv3H4D1twE2kDiXZLWdaBTIRM82iRdCi'
mpr = MPRester(api_key)

c:\cleen\Apps\Anaconda\lib\site-packages\mp_api\client\mprester.py:182: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


In [3]:
# possiblealloys = mpr.summary.search(theoretical=True,fields=["material_id","formula_pretty","decomposes_to","formation_energy_per_atom","energy_per_atom"])
realizedalloys = mpr.summary.search(theoretical=False, fields=["material_id","formula_pretty","decomposes_to","formation_energy_per_atom","energy_per_atom"])

Retrieving SummaryDoc documents:   0%|          | 0/49601 [00:00<?, ?it/s]

In [4]:
## Used to get data straight from MPR
writing = realizedalloys
writing_id = [oxide.material_id for oxide in writing]
writing_formula = [oxide.formula_pretty for oxide in writing]
writing_decompose = [oxide.decomposes_to for oxide in writing]
writing_formationenergyperatom = [oxide.formation_energy_per_atom for oxide in writing]
writing_energyperatom = [oxide.energy_per_atom for oxide in writing]
names_fixed = []
for alloy in writing_formula:
    transformed_text = chemparse.parse_formula(alloy)
    keys = np.array(list(transformed_text.keys()))
    values = np.array(list(transformed_text.values()))

    # Reshape the keys and values arrays to have two columns
    keys = np.reshape(keys, (-1, 1))
    values = np.reshape(values, (-1, 1))

    # Concatenate the keys and values arrays horizontally
    result = np.concatenate((keys, values), axis=1)

    output_string = ''
    for row in result:
        output_string += row[0] + row[1]
    names_fixed.append(output_string)


writing_data = np.column_stack((writing_id, writing_formula, writing_formationenergyperatom, writing_energyperatom))
writing_data = np.savetxt('realizedalloys_data.csv', writing_data, delimiter=',', fmt='%s')


In [5]:
alloynames = names_fixed
alloyformE = writing_formationenergyperatom

In [2]:
material_dataset = []

mainingredient = ['Superconductor'] ###
calculation1 = ['Formation Energy of supercon']
calculation2 = ['Weighted formation energy of likely oxides']

maincomp = ['atomicradius1','atomicradius2','atomicradius3','atomicradius4','atomicradius5','atomicradius6','atomicradius7','atomicradius8','atomicradius9','atomicradius10',
            'fusionheat1','fusionheat2','fusionheat3','fusionheat4','fusionheat5','fusionheat6','fusionheat7','fusionheat8','fusionheat9','fusionheat10',
            'thermalcond1','thermalcond2','thermalcond3','thermalcond4','thermalcond5','thermalcond6','thermalcond7','thermalcond8','thermalcond9','thermalcond10',
            'numvalence1','numvalence2','numvalence3','numvalence4','numvalence5','numvalence6','numvalence7','numvalence8','numvalence9','numvalence10',
            'Tc'] ###

comp1 = ['number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']
comp2 = ['number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']
comp3 = ['number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']
comp4 = ['number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']
comp5 = ['number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']
comp6 = ['number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']
comp7 = ['number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']
comp8 = ['number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']
comp9 = ['number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']
ingredient_coefficients = ['comp1','comp2','comp3','comp4','comp5','comp6','comp7','comp8','comp9']
all_comp = np.concatenate((ingredient_coefficients,comp1,comp2,comp3,comp4,comp5,comp6,comp7,comp8,comp9))


### Insert ingredients (materials) of interest here
mainingredient = np.array(['Cu1.0P6.0Pb9.0O25.0'])
calculation1 = np.array([-2.134*41])
maincomp = np.zeros((len(mainingredient),len(maincomp)))

In [3]:
# Boot up materials project
# api_key = 'Lv3H4D1twE2kDiXZLWdaBTIRM82iRdCi'
# mpr = MPRester(api_key)
binaryoxides = np.genfromtxt('binaryoxide_data.csv', delimiter=',', dtype=None, encoding=None)

#### OBTAIN calculation1
def getbinaryoxides(element):
    oxides_data = np.asarray([999999,999999,999999])
    checker = np.asarray([999999,999999,999999])
    chars = len(element)
    for oxides in binaryoxides:
        if element == oxides[1][0:chars] and oxides[1][chars:chars+1].isdigit():
            oxidename = oxides[1]
            oxideformE = oxides[2]
            oxideEperatom = oxides[3]
            oxides_data = np.vstack((oxides_data, [oxidename, oxideformE, oxideEperatom]))
    
    if np.shape(oxides_data) == np.shape(checker):
        return [0]
    else:
        rank_indices = np.argsort(oxides_data[:,1])
        rankedoxides = np.flip(oxides_data[rank_indices], axis=0)
        return rankedoxides

### End output here is ranked_neg_oxides with the most likely oxide first.
def weight_exp(arr):
    weighted_arr = []
    # Weight using a logistic function
    supremum = np.max(arr)
    exp = len(arr) / 2
    x0 = np.average(arr) - np.std(arr)
    for energies in arr:
        weight = supremum/(1+np.exp(exp*(energies-x0))) * -1
        weighted_arr.append(weight)    
        
    return weighted_arr

#### STORE calculation 1


In [4]:
### Create list of element oxides and their formation energies + weights
periodictable = pd.read_csv('PeriodicTableCleaned_data.csv')

elements = np.asarray(periodictable['symbol'])
elements_data = np.asarray(periodictable.iloc[:, 1:])
element_oxides = []

for sample in elements:
    test = getbinaryoxides(sample) 
    if np.shape(test) == np.shape([0]):
        element_oxides.append([0])
        continue
    list = test
    neg_oxides_name = []
    neg_oxides_formE = []
    neg_oxides_Eperatom = []

    ### Get rid of positive formation energy oxides.
    for oxides in list:
        if float(oxides[1]) < 0:
            neg_oxides_name.append(oxides[0])
            neg_oxides_formE.append(oxides[1])
            neg_oxides_Eperatom.append(oxides[2])

    neg_oxides = np.column_stack((neg_oxides_name, neg_oxides_formE, neg_oxides_Eperatom))
            
    uniques = np.unique(neg_oxides[:,0])
    list_uniques = []
    for oxides in uniques:
        formationE = neg_oxides[neg_oxides[:, 0] == oxides]
        likelyoxide = [formationE[0][0], formationE[0][1], formationE[0][2]]
        list_uniques.append(likelyoxide)
        # make into numpy array
    list_uniques = np.asarray(list_uniques)
    if len(list_uniques) == 0:
        element_oxides.append([0])
        continue
        
    ### list_uniques is the thing we want
    rank_list = np.argsort(list_uniques[:,1])
    ranked_neg_oxides = np.flip(list_uniques[rank_list], axis=0)
    ranked_neg_oxides_formE = np.asarray(ranked_neg_oxides[:,1], dtype = float)
    
    raw_weights_oxides = weight_exp(ranked_neg_oxides_formE)
    normalized_weights = raw_weights_oxides / np.sum(raw_weights_oxides)
    
    ####### THIS PART MAY NEED UPDATING IF WE DECIDE TO INCLUDE TERNARY OXIDES
    coeff_first = []
    coeff_second = []
    for oxides in ranked_neg_oxides:
        formula = oxides[0]
        coeff_first.append(re.search(r'\d+', formula).group())
        if formula[-1].isdigit():
            coeff_second.append(re.findall(r'\d+', formula)[1])
        else:
            coeff_second.append(1)
    
    coeff_first = np.asarray(coeff_first, dtype=float)
    coeff_second = np.asarray(coeff_second, dtype=float)
    netformE = np.asarray(ranked_neg_oxides[:,1], dtype=float) * (coeff_first + coeff_second)
    
    ### likely_oxides => Formatted as [Oxide name, Formation Energy / atom, Energy per atom, net formE, Weights, numfirstatoms]
    likely_oxides = np.column_stack((ranked_neg_oxides, netformE, normalized_weights, coeff_first))
    element_oxides.append(likely_oxides)
    
### Using this function takes the approach - 
## Assume that X% of element is allocated to this alloy, X% to the other alloy, X% to another...
# Then net formation energy is actual # of alloy made * net formation energy of alloy 

def componentformE_mass(element, size):
    indexarr = np.where(elements == element)
    index = indexarr[0][0]
    oxides = element_oxides[index]
    if oxides[0] == 0:
        return 0
    numelements_inoxide = np.asarray(oxides[:,5], dtype = float)
    weights = np.asarray(oxides[:,4], dtype = float)
    netformE = np.asarray(oxides[:,3], dtype = float)
    
    allocated_element = size * weights
    weighted_oxides = allocated_element / numelements_inoxide
    formE_oxides = weighted_oxides * netformE
    
    partialformE = np.sum(formE_oxides)
    return partialformE

### The other option is that X% of the TOTAL oxides made is 1, X% of the total is 2, ...
## Procedure is:
# First, based on coeff_first of the alloy, determine how much total hydrogen is required from percentages.
# So if there's 100 alloys total, X% * coeff_first[0] + X% * coeff_first[1] + ... = net # elements
#

def componentformE_total(element,size):
    indexarr = np.where(elements == element)
    index = indexarr[0][0]
    oxides = element_oxides[index]
    if oxides[0] == 0:
        return 0
    numelements_inoxide = np.asarray(oxides[:,5], dtype = float)
    weights = np.asarray(oxides[:,4], dtype = float)
    netformE = np.asarray(oxides[:,3], dtype = float)
    
    netelement = np.sum(weights*numelements_inoxide)
    multiplier = float(size) / float(netelement)
    formE_oxides = multiplier*netformE*weights
    
    partialformE = np.sum(formE_oxides)
    return partialformE

### Remember the formation energies of the oxides (weighted) is calculation2
calculation2 = []

import chemparse

for ingredient in mainingredient:
    parsedformula = chemparse.parse_formula(ingredient)
    parsedelements = [*parsedformula.keys()]
    parsedcomp = [*parsedformula.values()]
    parsed_main = np.column_stack((parsedelements,parsedcomp))
    calc_formE = []
    for components in parsed_main:
        element = components[0]
        size = components[1]
        formE = componentformE_total(element, size)
        calc_formE.append(formE)
    weightedformE = np.sum(np.asarray(calc_formE, dtype=float))
    calculation2.append(weightedformE)
    
### after running, calculation2 is now set. 

C:\Users\songo\AppData\Local\Temp\ipykernel_1724\1381960302.py:97: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if oxides[0] == 0:


In [5]:
### Now populate the component data. We leave those bois blank if there aren't enough components.T
all_comp = []
for ingredient in mainingredient:
    parsedformula = chemparse.parse_formula(ingredient)
    parsedelements = [*parsedformula.keys()]
    parsedcomp = [*parsedformula.values()]
    parsed_main = np.column_stack((parsedelements,parsedcomp))
    
    complen = len(comp1)*9
    compfill = len(parsedelements)*13
    coefflen = len(ingredient_coefficients)
    coeff_fill = len(parsedelements)
    comp_unfilled = complen - compfill
    coeff_unfilled = coefflen - coeff_fill
    
    unfilled_comp = np.zeros(comp_unfilled)
    unfilled_coeff = np.zeros(coeff_unfilled)
    filled_comp = []
    filled_coeff = []
    for parsed in parsed_main:
        element = parsed[0]
        weight = [parsed[1]]
        index = np.where(elements == element)[0][0]
        elementdata = elements_data[index]

        filled_coeff = np.concatenate((filled_coeff, weight))
        filled_comp = np.concatenate((filled_comp, elementdata))
        
    net_comp = np.concatenate((filled_comp, unfilled_comp))
    net_coeff = np.concatenate((filled_coeff, unfilled_coeff))
    
    row = np.concatenate((net_coeff, net_comp))
    
    all_comp.append(row)
    

In [6]:
## test - all first shape elements should be the same
print(np.shape(mainingredient))
print(np.shape(calculation1))
print(np.shape(calculation2))
print(np.shape(maincomp))
print(np.shape(all_comp))

(1,)
(1,)
(1,)
(1, 41)
(1, 126)


In [7]:
material_dataset_data = np.column_stack((mainingredient, calculation1, calculation2, maincomp, all_comp))

In [9]:
np.savetxt('predictionsofinterest.csv', material_dataset_data, delimiter=',',fmt='%s')
